In [1]:
type statistika
    algoritam
    vrijednosti
    vremena
    broj
    sumValue::Float64
    sumTime::Float64
    avgValue::Float64
    avgTime::Float64
end

function statistika(algoritam)
    vrijednosti = Array(Float64,0)
    vremena = Array(Float64,0)
    return statistika(algoritam,vrijednosti,vremena,0,0,0,0,0)
end

function dodaj!(sta::statistika,value,vrijeme)
    push!(sta.vrijednosti,value)
    push!(sta.vremena,vrijeme)
    sta.broj+=1
    sta.sumValue+=value
    sta.sumTime+=vrijeme
    sta.avgValue = sta.sumValue/sta.broj
    sta.avgTime = sta.sumTime/sta.broj
end

dodaj! (generic function with 1 method)

In [2]:
dir = pwd()
cd("$dir\\input")

type parr
    n::Int
    m::Int
    rjesenje::Int
end

function openNth(br,sadrzaj)
    filein = open(sadrzaj[br])
    puni=""
    for line in eachline(filein)
        puni = puni * line
    end
    puni = strip(puni)
    #println(puni)
    close(filein)
    return puni
end


function parsiraj!(puni,c,A,b,par::parr)
    brojevi_str = split(puni,r" |\r|\n",keep=false)
    par.n = parse(Int,brojevi_str[1])
    par.m = parse(Int,brojevi_str[2])
    par.rjesenje = parse(Int,brojevi_str[3])
    
    cA = brojevi_str[4:(3+par.n)]
    tempA = brojevi_str[4+par.n:(3+par.n+par.n*par.m)]
    bA = brojevi_str[4+par.n+par.n*par.m:(3+par.n+par.n*par.m+par.m)]
    
    for i in bA
        ii = parse(Float64,i)
        push!(b,ii)
    end
    for i in cA
        ii = parse(Float64,i)
        push!(c,ii)
    end
    
    for poc in 1:par.n:(par.m*par.n)
        #println(poc)
        tt = []
        for i in 0:(par.n-1)
            #print(tempA[poc+i]*" ")
            push!(tt,parse(Float64,tempA[poc+i]))
        end

        push!(A,map(Float64,tt))
    end
end

parsiraj! (generic function with 1 method)

In [3]:
type Individua
    Duzina
    Hromozom
    Fitnes::Float64
end
function Individua(Duzina::Int)
    Hromozom = BitArray(Duzina)
    rand!(Hromozom)
    return Individua(Duzina,Hromozom,0.0)
end
function Individua(Hromozom::BitArray)
    Duzina = length(Hromozom)
    return Individua(Duzina,Hromozom,0.0)
end

Individua

In [4]:
function Mutiraj!(a::Individua)
    poz = rand(1:a.Duzina)
    a.Hromozom[poz] = !a.Hromozom[poz]
    return a
end
function Ukrstanje(a::Individua,b::Individua,p::Float64)
    tacka1 = rand(1:(a.Duzina-1))
    tacka2 = rand(1:(a.Duzina-1))
    if(tacka2<tacka1)
        temp = tacka2
        tacka2 = tacka1
        tacka1 = temp
    end
    #println(tacka)
    t1 = Individua([a.Hromozom[1:tacka1];b.Hromozom[(tacka1+1):tacka2];a.Hromozom[(tacka2+1):end]])
    t2 = Individua([b.Hromozom[1:tacka1];a.Hromozom[(tacka1+1):tacka2];b.Hromozom[(tacka2+1):end]])
    if(p>rand())
        return t1
    end
    return t2
end
function Ukrstanje(a::Individua,b::Individua)
    tacka1 = rand(1:(a.Duzina-1))
    tacka2 = rand(1:(a.Duzina-1))
    if(tacka2<tacka1)
        temp = tacka2
        tacka2 = tacka1
        tacka1 = temp
    end
    #println(tacka)
    t1 = Individua([a.Hromozom[1:tacka1];b.Hromozom[(tacka1+1):tacka2];a.Hromozom[(tacka2+1):end]])
    t2 = Individua([b.Hromozom[1:tacka1];a.Hromozom[(tacka1+1):tacka2];b.Hromozom[(tacka2+1):end]])

    return [t1;t2]
end
function UkrstanjeUn(a::Individua,b::Individua)
    rez1=BitArray(a.Duzina)
    rez2=BitArray(a.Duzina)
    
    for i in 1:a.Duzina
        p = rand()
        
        if(p<0.5)
            rez1[i]=a.Hromozom[i]
            rez2[i]=b.Hromozom[i]
        else
            rez2[i]=a.Hromozom[i]
            rez1[i]=b.Hromozom[i]
        end
    end
    
    return [Individua(rez1);Individua(rez2)]
end

function UkrstanjeN(a::Individua,b::Individua,p::Float64,n::Int)
    tacke = Array(Int,0)
    niz = 2:(a.Duzina-1)
    #println(niz)
    for s in 1:n
        t = rand(niz)
        push!(tacke,t)
    end
    sort!(tacke)

    kraj = length(tacke)
    h1 = [b.Hromozom[1:tacke[1]]]
    h2 = [a.Hromozom[1:tacke[1]]]
    #println(h1)
    for i in 1:2:(kraj-1)
        
        if((i+1)>kraj||(i+2)>kraj)
            break
        end
        #println("tacke[$i] = $(tacke[i]), tacke[$i+1] = $(tacke[i+1]), tacke[$i+2] = $(tacke[i+2])")
        #a.Hromozom[tacka[i]:tacka[i+1]];b.Hromozom[(tacka[i+1]+1):tacka[i+2]]
        temp = h1
        h1 = [temp;a.Hromozom[(tacke[i]+1):tacke[i+1]];b.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
        h2 = [temp;b.Hromozom[(tacke[i]+1):tacke[i+1]];a.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
    end
    
    h1 = [h1;b.Hromozom[(tacke[kraj]+1):end]]
    h2 = [h2;a.Hromozom[(tacke[kraj]+1):end]]
    
    t1 = Individua(h1)
    t2 = Individua(h2)    
    
    if(p>rand())
        return t1
    end
    return t2
end
function UkrstanjeN(a::Individua,b::Individua,n::Int)
    tacke = Array(Int,0)
    niz = 2:(a.Duzina-1)
    #println(niz)
    for s in 1:n
        t = rand(niz)
        push!(tacke,t)
    end
    sort!(tacke)

    kraj = length(tacke)
    h1 = [b.Hromozom[1:tacke[1]]]
    h2 = [a.Hromozom[1:tacke[1]]]
    #println(h1)
    for i in 1:2:(kraj-1)
        
        if((i+1)>kraj||(i+2)>kraj)
            break
        end
        #println("tacke[$i] = $(tacke[i]), tacke[$i+1] = $(tacke[i+1]), tacke[$i+2] = $(tacke[i+2])")
        #a.Hromozom[tacka[i]:tacka[i+1]];b.Hromozom[(tacka[i+1]+1):tacka[i+2]]
        temp = h1
        h1 = [temp;a.Hromozom[(tacke[i]+1):tacke[i+1]];b.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
        h2 = [temp;b.Hromozom[(tacke[i]+1):tacke[i+1]];a.Hromozom[(tacke[i+1]+1):tacke[i+2]]]
    end
    
    h1 = [h1;b.Hromozom[(tacke[kraj]+1):end]]
    h2 = [h2;a.Hromozom[(tacke[kraj]+1):end]]
    
    t1 = Individua(h1)
    t2 = Individua(h2)    
    
    return [t1;t2]
end

function UInt(niz)#ime ostalo xD
    rez=Array(Float64,0)
    for a in niz
        push!(rez,Float64(a))
    end
    return rez;
end
function Evauliraj!(a::Individua,A,c,b,n,m)
    #a.Fitnes = rand()
    tt=UInt(a.Hromozom)
    fitnes = tt'*c
    kazna = fitnes
    for i in 1:m
        sum = 0.0
        for j in 1:n
            sum += (A[i][j]*tt[j])
        end
        if sum<=b[i]
            #continue
        else
            fitnes-=kazna
        end
    end
    #println(fitnes)
    a.Fitnes = fitnes[1]
end
function Evauliraj!(a::Individua)
    a.Fitnes = rand()
end

Evauliraj! (generic function with 2 methods)

In [5]:
type Populacija
    DuzinaHromozoma::Int
    MaxGeneracija::Int
    VelicinaPopulacije::Int
    BrElite::Int
    BrMutacija::Int
    Iteracija::Int
    Populacija
end
function Populacija(Duzina::Int64)
    MaxGeneracija = 100*Duzina # 100 * brojvarijabli
    VelicinaPopulacije = 51 #neparn broj uvijek zbog neceg
    BrElite = 1;
    BrMutacija = 15;
    Iteracija = 0;
    populacija = [];
    for a in 1:VelicinaPopulacije
        push!(populacija,Individua(Duzina))
    end
    return Populacija(Duzina,MaxGeneracija,VelicinaPopulacije,BrElite,BrMutacija,Iteracija,populacija)
end
function Populacija(Duzina::Int64,VelicinaPopulacije::Int64,BrMutacija::Int64)
    MaxGeneracija = 100*Duzina # 100 * brojvarijabli
    BrElite = 1;
    Iteracija = 0;
    populacija = [];
    for a in 1:VelicinaPopulacije
        push!(populacija,Individua(Duzina))
    end
    return Populacija(Duzina,MaxGeneracija,VelicinaPopulacije,BrElite,BrMutacija,Iteracija,populacija)
end

Populacija

In [6]:
function EvalPop!(pop::Populacija,m,n,A,b,c)
    for a in pop.Populacija
        Evauliraj!(a,A,c,b,n,m)
    end
    sort!(pop.Populacija, by=x->(x.Fitnes), rev=true)
end
function EvalPop!(pop::Populacija)
    for a in pop.Populacija
        Evauliraj!(a)
    end
    sort!(pop.Populacija, by=x->(x.Fitnes), rev=true)
end
function Turnir(pop::Populacija,m::Int64,p::Float64)
    rez = [];
    for a in 1:m
        temp = [];
        t = rand(1:pop.VelicinaPopulacije)
        push!(temp,pop.Populacija[t])
        t = rand(1:pop.VelicinaPopulacije)
        push!(temp,pop.Populacija[t])
        sort!(temp,by=x->(x.Fitnes))
        if(p<rand())
            push!(rez,temp[1])
        else
            push!(rez,temp[2])
        end
    end
    return rez
end
function Turnir(pop::Populacija,p::Float64)
    return Turnir(pop,2,p);
end
function RWS(pop::Populacija)
    p = Array(Float64,pop.VelicinaPopulacije)
    suma = 0;
    for i in 1:pop.VelicinaPopulacije
        suma+=pop.Populacija[i].Fitnes
    end
    max = 0.99
    for i in pop.VelicinaPopulacije:-1:1
        if(pop.Populacija[i].Fitnes<0)
            p[i] = max
            max-=0.01
        else
            p[i]=abs(pop.Populacija[i].Fitnes/suma)
        end
    end    
    j = 1;
    suma = p[j]
    u = rand()
    while (suma<u)
        j+=1
        suma+=(p[j])
    end
    return pop.Populacija[j]
end


RWS (generic function with 1 method)

In [7]:
function DajElite(pop::Populacija)
    br = pop.BrElite
    return pop.Populacija[1:br]
end
function Mutacija(pop::Populacija,djeca)
    for i in 1:pop.BrMutacija
        br = rand(1:length(djeca))
        #println("$i = $br")
        d = djeca[br]
        t = Mutiraj!(d)
        djeca[br] = t;
        #print(".")
    end
end
function ProsjekFitnesa(pop::Populacija)
    suma = 0;
    for a in pop.Populacija 
        suma+=a.Fitnes 
    end
    suma/=length(pop.Populacija)
    return suma
end

ProsjekFitnesa (generic function with 1 method)

In [8]:
function DajDjecu(pop::Populacija,roditelji)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece
        r1 = roditelji[rand(1:length(roditelji))]
        r2 = roditelji[rand(1:length(roditelji))]
        #d = UkrstanjeN(r1,r2,11)
        d = Ukrstanje(r1,r2)
        #println("d: $d \n\n")
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end
function DajDjecu(pop::Populacija)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece
        roditelji = Turnir(pop,0.9)
        r1 = roditelji[1]
        r2 = roditelji[2]
        d = Ukrstanje(r1,r2)
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end
function DajDjecuUn(pop::Populacija)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece
        roditelji = Turnir(pop,0.9)
        r1 = roditelji[1]
        r2 = roditelji[2]
        d = UkrstanjeUn(r1,r2)
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end
function DajDjecuUnRWS(pop::Populacija)
    rez = []
    brDjece = pop.VelicinaPopulacije - pop.BrElite
    while length(rez)<=brDjece        
        r1 = RWS(pop)
        r2 = RWS(pop)
        d = UkrstanjeUn(r1,r2)
        push!(rez,d[1])
        push!(rez,d[2])
    end
    return rez
end

function kombo1(m,n,A,b,c,rjesenje)
    pop = Populacija(n)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija
        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
            #println("kombo1: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n\n")
        end
                      
        #if(naj==rjesenje)
            #println("kombo1: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n\n")
            #break;
        #end

        #println("- populacija $pop\n")

        elite = DajElite(pop)

        #println("- elite $elite\n")

        roditelji = Turnir(pop,Int((pop.VelicinaPopulacije-pop.BrElite)/2),0.9)

        #println("- roditelji $roditelji\n")

        djeca = DajDjecu(pop,roditelji)

        #println("- djeca $djeca\n")

        Mutacija(pop,djeca)

        #println("- djeca $djeca\n")

        #novagen = 
        append!(djeca,elite)

        #println("- nova gen $djeca\n")
        
        pop.Populacija = djeca

    
        #println(pop.Iteracija)
        #break
    end
    return naj
end

kombo1 (generic function with 1 method)

In [9]:
function kombo2(m,n,A,b,c,rjesenje)
    pop = Populacija(n)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
            #println("kombo2: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n")
            break;
        end                      
        #if(naj==rjesenje)
        #    println("kombo2: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n")
            #break;
        #end

        elite = DajElite(pop)
        djeca = DajDjecu(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo2 (generic function with 1 method)

In [10]:
function kombo3(m,n,A,b,c,rjesenje)
    pop = Populacija(n)
    naj=0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
           # println("kombo3: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n")
            break;
        end                      
        #if(naj==rjesenje)
        #    println("kombo3: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n")
            #break;
        #end

        elite = DajElite(pop)
        djeca = DajDjecuUn(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo3 (generic function with 1 method)

In [11]:
function kombo3_1(m,n,A,b,c,rjesenje)
    pop = Populacija(n,21,1)
    #(Duzina::Int64,VelicinaPopulacije::Int64,BrMutacija::Int64)
    naj=0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
            #println("kombo3.1: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n")
            break;
        end                      
        #if(naj==rjesenje)
        #    println("kombo3: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n")
            #break;
        #end

        elite = DajElite(pop)
        djeca = DajDjecuUn(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end
function kombo2_1(m,n,A,b,c,rjesenje)
    pop = Populacija(n,21,1)
    #(Duzina::Int64,VelicinaPopulacije::Int64,BrMutacija::Int64)
    naj=0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
            #println("kombo2.1: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n")
            break;
        end                      
        #if(naj==rjesenje)
        #    println("kombo3: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n")
            #break;
        #end

        elite = DajElite(pop)
        djeca = DajDjecu(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo2_1 (generic function with 1 method)

In [12]:
function kombo3_2(m,n,A,b,c,rjesenje)
    pop = Populacija(n,51,1)
    #(Duzina::Int64,VelicinaPopulacije::Int64,BrMutacija::Int64)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
            #println("kombo3.2: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n")
            break;
        end                      
        #if(naj==rjesenje)
        #    println("kombo3: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n")
            #break;
        #end

        elite = DajElite(pop)
        djeca = DajDjecuUn(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end
function kombo2_2(m,n,A,b,c,rjesenje)
    pop = Populacija(n,51,1)
    #(Duzina::Int64,VelicinaPopulacije::Int64,BrMutacija::Int64)
    naj = 0
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
            #println("kombo2.2: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n")
            break;
        end                      
        #if(naj==rjesenje)
        #    println("kombo3: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n")
            #break;
        #end

        elite = DajElite(pop)
        djeca = DajDjecu(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end
function kombo4_1(m,n,A,b,c,rjesenje)
    pop = Populacija(n,51,1)
    naj = 0
    #(Duzina::Int64,VelicinaPopulacije::Int64,BrMutacija::Int64)
    
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
            #println("kombo4.1: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n")
            break;
        end                      
        #if(naj==rjesenje)
        #    println("kombo3: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n")
            #break;
        #end

        elite = DajElite(pop)
        djeca = DajDjecuUnRWS(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo4_1 (generic function with 1 method)

In [13]:
function kombo4(m,n,A,b,c,rjesenje)
    pop = Populacija(n)#,51,1)
    naj = 0
    #(Duzina::Int64,VelicinaPopulacije::Int64,BrMutacija::Int64)
    
    for pop.Iteracija in 1:pop.MaxGeneracija        
        EvalPop!(pop,m,n,A,b,c)

        prosjek = ProsjekFitnesa(pop)
        naj = pop.Populacija[1].Fitnes
        
        if(pop.Iteracija==pop.MaxGeneracija)
            #println("kombo4.1: Prosjek fitnesa $prosjek, najbolji fitnes $naj\n")
            break;
        end                      
        #if(naj==rjesenje)
        #    println("kombo3: Prosjek fitnesa $prosjek, najbolji fitnes $naj, iteracija $(pop.Iteracija)\n")
            #break;
        #end

        elite = DajElite(pop)
        djeca = DajDjecuUnRWS(pop)
        Mutacija(pop,djeca)
        append!(djeca,elite)
        pop.Populacija = djeca
    end
    return naj
end

kombo4 (generic function with 1 method)

In [14]:
sadrzaj = readdir()

stKombo1 = statistika("kombo1")
stKombo2 = statistika("kombo2")
stKombo2_1 = statistika("kombo2_1")
stKombo2_2 = statistika("kombo2_2")
stKombo3 = statistika("kombo3")
stKombo3_1 = statistika("kombo3_1")
stKombo3_2 = statistika("kombo3_2")
stKombo4 = statistika("kombo4")
stKombo4_1 = statistika("kombo4_1")

tic()
for file in 1:50
    print(".");
    #println("-- $file --")
    puni = openNth(1,sadrzaj)
    c = Array(Float64,0); b= Array(Float64,0)
    A = []
    par = parr(0,0,0)
    parsiraj!(puni,c,A,b,par)
    n = par.n; m= par.m; rjesenje = par.rjesenje

    #println("n = $(n) broj objekata, dimenzije hromozoma\nm = $(m) broj dimenzija\nrjesenje = $rjesenje\n")     


    
    tic()
    value = kombo1(m,n,A,b,c,rjesenje)
    dodaj!(stKombo1,value,toq())
    #toq()
    tic()
    value = kombo2(m,n,A,b,c,rjesenje)
    dodaj!(stKombo2,value,toq())
    #toc()
    tic()
    value = kombo3(m,n,A,b,c,rjesenje)
    dodaj!(stKombo3,value,toq())
    #toc()
    
    tic()
    value = kombo3_1(m,n,A,b,c,rjesenje)
    dodaj!(stKombo3_1,value,toq())
    #toc()
    tic()
    value = kombo2_1(m,n,A,b,c,rjesenje)
    dodaj!(stKombo2_1,value,toq())
    #toc()
    tic()
    value = kombo3_2(m,n,A,b,c,rjesenje)
    dodaj!(stKombo3_2,value,toq())
    #toc()
    tic()
    value = kombo2_2(m,n,A,b,c,rjesenje)
    dodaj!(stKombo2_2,value,toq())
    #toc()
    tic()
    value = kombo4(m,n,A,b,c,rjesenje)
    dodaj!(stKombo4,value,toq())
    
    tic()
    value = kombo4_1(m,n,A,b,c,rjesenje)
    dodaj!(stKombo4_1,value,toq())
    #toc()
    #break
end
toc()

println(stKombo1)
println(stKombo2)
println(stKombo2_1)
println(stKombo2_2)
println(stKombo3_1)
println(stKombo3_2)
println(stKombo4)
println(stKombo4_1)

..................................................elapsed time: 744.722079422 seconds
statistika("kombo1",[4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4005.0,4005.0,4015.0,4015.0,4015.0,4015.0,4015.0,4005.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0,4005.0,4015.0,4015.0,4015.0,4015.0,4015.0,4015.0],[1.627733251,1.304549149,1.329395751,1.340780556,1.29938625,1.317373571,1.320375863,1.42479659,1.394617884,1.318981603,1.321987444,1.320253594,1.327323887,1.317835432,1.319723499,1.365648062,1.844496148,1.44222108,2.133326655,1.359112597,1.352040332,1.414873489,1.357108177,1.44585917,1.446910681,1.47006879,1.369260514,1.441638134,1.420663115,1.568925103,1.566085706,1.35590442,1.369833206,1.364711325,1.403878762,1.463025317,2.542595893,1.628066532,1.568095253,1.516634279,1.475516852,1.551876491,1.484516626,1.607534441,1.65717361,1.447970